# Fantasy Pros Perfect Draft Analysis
### 01/21/22
#### Purpose: There is a challenge to use the 2021 preseason ADPs and 

#### Import Libraries

In [1]:
import pandas as pd
from typing import Optional

#### Read in data and clean a little

In [2]:
df = pd.read_csv('../yahoo/output/2021_player_week_points.csv').drop_duplicates()
df['week'] = df['game_id'].str[5:7]

In [9]:
adp_rankings = pd.read_csv('input/FantasyPros_2021_Overall_ADP_Rankings.csv')

#### Create functions

In [3]:
def add_player_bye_week(df:pd.DataFrame) -> pd.DataFrame:
    """
    Objective:
    ----------
    Takes a players weeks and points and adds a bye week with zero points
    Also works for games they may have missed due to injury
    
    Parameters:
    -----------
    df: pd.DataFrame
        Pandas dataframe with columns at least of 'week' and 'fantasy_points'
    
    Returns:
    --------
    final_df: pd.DataFrame
        Pandas dataframe that fills in missing week with a score of 0
    """
    zero_df = pd.DataFrame({'week': ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17'], 
                            'bye_points': [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]})
    
    new_df = pd.merge(df, zero_df, how='outer', on=['week'])
    
    new_df['fantasy_points'] = new_df[['fantasy_points', "bye_points"]].max(axis=1)
    
    final_df = new_df[['week','fantasy_points']].sort_values('week')
    
    return final_df

In [4]:
def get_player_points_list(df:pd.DataFrame,
                           player:str,
                           position:str) -> list:
    """
    Objective:
    ----------
    Take a dataframe, filters to a selected player, adds the bye week, and returns a sorted list of points
    
    Parameters:
    -----------
    df: pd.DataFrame
        Pandas dataframe with columns at least of 'player', 'week', and 'fantasy_points'
    
    Returns:
    --------
    player_points_list: list
        A sorted list of player fantasy points for each week of the season
    
    """
    player_points = df[(df.player == player) & (df.position == position)][['week','fantasy_points']]
    player_points_list = add_player_bye_week(player_points)['fantasy_points'].tolist()
    
    return player_points_list

In [5]:
def find_complimentary_player(df:pd.DataFrame, 
                              position:str,
                              comp_player:str,
                              comp_list:Optional[list] = []
                             ) -> str:
    """
    Objective:
    ----------
    Find the best complimentory player to a suggested player in the draft. Sorts through all players at the position or a selected list.
    
    Parameters:
    -----------
    df: pd.DataFrame
        Pandas datafame that has at a minimum: player name, week, fantasy points
    position: str
        Position of the player of interest which will determine the players to compare against
    comp_player: str
        The selected player that other players will be compared against
    comp_list: Optional[list]
        An optional list of players that are under consideration for comparison. 
        If optimal option is not available, this is a good work around but there is no error checking on names being correct
    
    Returns:
    --------
    final_output: str
        Printed results of the best player to select to compliment the chosen player
    
    """
    # sorting the values so the weeks are in order and filtering out week 18 that's not included
    df = df[df.week <= '17']
    
    # need to impute a 0 for the selected player bye week
    selected_player_points_list = get_player_points_list(df, comp_player, position)
    
    # error checking on selected player
    if len(df[df.player == comp_player]) == 0:
        raise ValueError('Player name not found in data')
    
    comp_player_data = df[df.position == position]
    # error checking on input position
    if len(comp_player_data) == 0:
        raise ValueError('Position not found in data')
    
    total_points = 0
    max_points = 0
    if len(comp_list) > 0:
        print('Comparison list given')
        comp_player_list = comp_list
        
    else:
        print('Comparison list not given')
        comp_player_list = comp_player_data[['player']].drop_duplicates()['player'].tolist()
    
    # loop through all players or selected list
    for player in comp_player_list:
        comp_player_points_list = get_player_points_list(df, player, position)
        
        #go week by week and take the maximum value between two players and sum
        for selected_points, comp_points in zip(selected_player_points_list, comp_player_points_list):
            max_val = max(selected_points, comp_points)
            total_points = total_points + max_val
        
        # if the comparison points are the highest, make it the max value, otherwise continue searching and reset the mark
        if total_points > max_points:
            max_points = total_points
            max_compliment_player = player
        total_points = 0
    
    return print('\nThe best compliment to ', comp_player, ' is: ', max_compliment_player, '\nTheir total points are: ', max_points, sep = '')
    

#### Examples

In [6]:
find_complimentary_player(df, 'TE', 'T.Kelce')

Comparison list not given

The best compliment to T.Kelce is: M.Andrews
Their total points are: 291.7


In [7]:
find_complimentary_player(df, 'TE', 'T.Kelce', comp_list = ['D.Knox','Z.Ertz'])

Comparison list given

The best compliment to T.Kelce is: D.Knox
Their total points are: 260.8
